In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [2]:
weather = pd.read_csv("weather3_180703.csv")
weather['date'] = pd.to_datetime(weather["date"])
station = weather[weather['station_nbr'] == 14]

In [3]:
station['date'].count()

1035

In [4]:
# ls == 모든 값이 0인 것

ls = []
ls1 = []
for i in station.columns:
    if station[i].count() == 0:
        ls.append(i)
    if station[i].isna().sum() != 0:
        ls1.append(i)
ls, ls1

([], ['depart', 'sunrise', 'sunset', 'resultdir', 'daytime'])

In [5]:
for i in station.columns:
    count_null = station[i].isna().sum()
    print(i, ":", count_null, "(",round((count_null / len(station) * 100),2),"%",")")

station_nbr : 0 ( 0.0 % )
date : 0 ( 0.0 % )
tmax : 0 ( 0.0 % )
tmin : 0 ( 0.0 % )
tavg : 0 ( 0.0 % )
depart : 31 ( 3.0 % )
dewpoint : 0 ( 0.0 % )
wetbulb : 0 ( 0.0 % )
heat : 0 ( 0.0 % )
cool : 0 ( 0.0 % )
sunrise : 31 ( 3.0 % )
sunset : 31 ( 3.0 % )
codesum : 0 ( 0.0 % )
snowfall : 0 ( 0.0 % )
preciptotal : 0 ( 0.0 % )
stnpressure : 0 ( 0.0 % )
sealevel : 0 ( 0.0 % )
resultspeed : 0 ( 0.0 % )
resultdir : 2 ( 0.19 % )
avgspeed : 0 ( 0.0 % )
year : 0 ( 0.0 % )
month : 0 ( 0.0 % )
day : 0 ( 0.0 % )
relative_humility : 0 ( 0.0 % )
windchill : 0 ( 0.0 % )
daytime : 31 ( 3.0 % )
week7 : 0 ( 0.0 % )
weekend : 0 ( 0.0 % )
code_change : 0 ( 0.0 % )
rainY : 0 ( 0.0 % )
otherY : 0 ( 0.0 % )
nothing : 0 ( 0.0 % )


In [6]:
train = pd.read_csv("train.csv")
train.date = pd.to_datetime(train.date)
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [7]:
key = pd.read_csv("key.csv")
station = station.merge(key)

In [8]:
station = station.merge(train)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,units
388495,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,10.766667,4,0,0,0,0,1,42,107,0
388496,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,10.766667,4,0,0,0,0,1,42,108,0
388497,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,10.766667,4,0,0,0,0,1,42,109,0
388498,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,10.766667,4,0,0,0,0,1,42,110,0
388499,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,10.766667,4,0,0,0,0,1,42,111,0


In [9]:
station['log1p_units'] = np.log1p(station.units)

In [10]:
target1 = station['units']
target2 = station['log1p_units']

station.drop(columns=['units','log1p_units'],inplace=True)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,windchill,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr
388495,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,44.019672,10.766667,4,0,0,0,0,1,42,107
388496,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,44.019672,10.766667,4,0,0,0,0,1,42,108
388497,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,44.019672,10.766667,4,0,0,0,0,1,42,109
388498,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,44.019672,10.766667,4,0,0,0,0,1,42,110
388499,14,2014-10-31,60.0,38.0,49.0,-7.0,31.0,42.0,16.0,0.0,...,44.019672,10.766667,4,0,0,0,0,1,42,111


In [11]:
df1 = pd.concat([station,target1], axis=1)
df2 = pd.concat([station,target2], axis=1)

In [12]:
df2.to_csv('station14.csv', sep=',', index=False)

In [13]:
if df2.columns.tolist() and ls:
    print('공통 변수 있음')
    print(df2.columns.tolist() and ls)
else:
    print('공통 변수 없음')

공통 변수 없음


In [14]:
df2.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'year', 'month', 'day', 'relative_humility', 'windchill',
       'daytime', 'week7', 'weekend', 'code_change', 'rainY', 'otherY',
       'nothing', 'store_nbr', 'item_nbr', 'log1p_units'],
      dtype='object')

In [15]:
ls, ls1

([], ['depart', 'sunrise', 'sunset', 'resultdir', 'daytime'])

### 1. 변수변환: df2 (log1p_units)

In [19]:
all_var = 'log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + C(daytime) + 0'

In [16]:
model2 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     3041.
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:42:41   Log-Likelihood:            -1.8333e+05
No. Observations:              388500   AIC:                         3.669e+05
Df Residuals:                  388357   BIC:                         3.685e+05
Df Model:                         142                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

### 2. 변수변환 : df2 (log1p_units) + 아웃라이어 제거

In [17]:
# 아웃라이어 제거
# Cook's distance > 2 인 값 제거
influence = result2.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (len(df2) - 2)
idx_outlier = np.where(cooks_d2 > fox_cr)[0]
len(idx_outlier)

14505

In [18]:
idx = list(set(range(len(df2))).difference(idx_outlier))
df2_1 = df2.iloc[idx, :].reset_index(drop=True)
df2_1

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,log1p_units
0,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,1,0.0
1,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,2,0.0
2,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,3,0.0
3,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,4,0.0
4,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,6,0.0
5,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,7,0.0
6,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,8,0.0
7,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,10,0.0
8,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,11,0.0
9,14,2012-01-01,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,...,9.833333,6,1,0,0,0,1,2,12,0.0


In [19]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                 4.268e+04
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:47:35   Log-Likelihood:             7.2459e+05
No. Observations:              373995   AIC:                        -1.449e+06
Df Residuals:                  373853   BIC:                        -1.447e+06
Df Model:                         141                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

In [20]:
ls1

['depart', 'sunrise', 'sunset', 'resultdir', 'daytime']

### 3. VIF 확인

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill', 'snowfall']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF Factor,features
0,inf,tmax
1,inf,tmin
2,inf,tavg
3,1.042388e+06,sealevel
4,1.032478e+06,stnpressure
5,9.559696e+03,windchill
6,2.891425e+03,wetbulb
7,2.036382e+03,dewpoint
8,5.289556e+02,relative_humility
9,1.306042e+02,heat


### 4. VIF 근거 변수 빼고 다시 Model 돌림.

In [22]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(cool)\
                            + scale(snowfall) + scale(preciptotal) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                 4.559e+04
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        12:13:24   Log-Likelihood:             7.2458e+05
No. Observations:              373995   AIC:                        -1.449e+06
Df Residuals:                  373862   BIC:                        -1.447e+06
Df Model:                         132                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0002      0

### 5. VIF 재확인 (tmax, tmin, tavg 제외함)

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['cool', 'snowfall', 'preciptotal', 'resultspeed', 'avgspeed']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

,VIF Factor,features
0,17.770309,avgspeed
1,17.474920,resultspeed
2,1.372002,cool
3,1.094665,preciptotal
4,1.033673,snowfall


### 6. VIF 근거 avg speed, result speed 제외하고  Model 돌림.

In [24]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(cool)\
                            + scale(snowfall) + scale(preciptotal) \
                            + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                 4.629e+04
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        12:19:07   Log-Likelihood:             7.2458e+05
No. Observations:              373995   AIC:                        -1.449e+06
Df Residuals:                  373864   BIC:                        -1.447e+06
Df Model:                         130                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0002      0

### Anova 분석을 통해 p-value가 0.05보다 큰것은 제외

In [25]:
anova_result2_1 = sm.stats.anova_lm(result2_1).sort_values(by=['PR(>F)'], ascending = False)
anova_result2_1[anova_result2_1['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(rainY),1.0,0.044425,0.044425,36.539267,1.497624e-09
C(weekend),1.0,0.638783,0.638783,525.399489,3.405796e-116
C(month),11.0,0.940881,0.085535,70.352291,1.136054e-158
C(year),3.0,20.306340,6.768780,5567.325115,0.000000e+00
C(store_nbr),3.0,6.087030,2.029010,1668.861688,0.000000e+00
C(item_nbr),109.0,7307.972801,67.045622,55145.059923,0.000000e+00


### 최종 Anova 분석과 TIF 통과한 변수들을 바탕으로 최종 모델생성

In [27]:
# OLS - df2_1_1
model2_1 = sm.OLS.from_formula('log1p_units ~ C(year) + C(month) + \
C(weekend) + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)

result = model2_1.fit()
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                 4.738e+04
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        12:22:06   Log-Likelihood:             7.2458e+05
No. Observations:              373995   AIC:                        -1.449e+06
Df Residuals:                  373867   BIC:                        -1.448e+06
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0003      0

In [28]:
from patsy import dmatrix
# 독립변수와 종속변수로 나누기
df2_1_X = df2_1.drop(columns=['log1p_units'])
df2_1_target = df2_1['log1p_units']

In [29]:
formula = 'C(year) + C(month) + C(weekend) \
        + C(rainY) + C(store_nbr) + C(item_nbr) + 0'
dfX = dmatrix(formula, df2_1_X, return_type='dataframe')
dfy = pd.DataFrame(df2_1_target, columns=["log1p_units"])

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(dfX)):
    X_train = dfX.values[train_index]
    y_train = dfy.values[train_index]
    X_test = dfX.values[test_index]
    y_test = dfy.values[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.91234521, 0.88995094, 0.86880832, 0.93505502, 0.90409244,
       0.92914693, 0.94627414, 0.96090479, 0.97230464, 0.97701019])